#API returning predicted house price in King County, USA in 2015.

**Features:** 
 
 sqft_lot - Square footage of the lot (median: 7863.0) 


long- Longitude coordinate (median: -122.218) 


yr_built - Year when house was built (median: 1974) 


sqft_above - Square footage of house (apart from basement) (median: 1520.0) 


zipcode - Zip code in which house is located (median: 98058) 


sqft_living15 - The square footage of interior housing living space for the nearest 15 neighbors (median: 1830.0) 

grade - Overall grade given to the housing unit, based on King County grading system (range: 0-12) 


sqft_living - Square footage of the house (median: 1890.0) 

lat - Latitude coordinate (median: 47.561) 

price - Price (in hundreds of thousands of USD) is prediction target (median: 43) 

In [103]:
import numpy as np

import joblib

!pip install flask-ngrok
from flask import Flask
from flask_ngrok import run_with_ngrok

# import warnings
# warnings.filterwarnings("ignore", category=UserWarning)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [104]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Colab Notebooks/Flask/HousePricing

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks/Flask/HousePricing


In [105]:
model =joblib.load('baggregr_model.joblib')
scaler = joblib.load('baggregr_scaler.pkl')

In [106]:
#[3'sqft_lot', 15'long', 11'yr_built', 9'sqft_above', 13'zipcode',
#       16'sqft_living15', 8'grade', 2'sqft_living', 14'lat']
def return_predictions(model, scaler, features):

    X = np.zeros(18)
    X[15] = -122.218
    X[9] = float(features['sqft_living'])*1520.0/1890.0
    X[13] = 98058.0
    X[16] = 1830.0
    X[14] = 47.561
    X[[3, 2, 11, 8]] = list(features.values())
    
    data = scaler.transform(X.reshape(1, -1))
    pred = model.predict(data[:, [3, 15, 11,  9, 13, 16,  8,  2, 14]])
    data[:, 17] = pred

    return scaler.inverse_transform(data)[0, 17]

#Flask app

In [107]:
from flask import Flask, request, jsonify

app = Flask(__name__)
app.config['SECRET_KEY'] = 'ghfgdgs55rgfr87sr'
run_with_ngrok(app)

@app.route('/', methods=['POST'])
def home():

  content = request.json
    
  predicted = return_predictions(model, scaler, content)*10000
  
  #return json.loads(last12), json.loads(predicted)
  return jsonify({'estimated price': predicted})

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://75ba-35-236-176-20.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
INFO:werkzeug:127.0.0.1 - - [09/Oct/2022 14:17:43] "POST / HTTP/1.1" 200 -
